In [ ]:
#Importing needed modules.
from sklearn.metrics import classification_report # used for generating a classification report
from os import mkdir , listdir
from shutil import copyfile , rmtree #to copy files and remove directories.
from random import sample
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers , Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import InceptionV3#InceptionV3 is a popular and powerful pre-trained convolutional neural network (CNN) architecture designed by Google for image classification tasks
import matplotlib.pyplot as plt
#Instancing from 'ImageDataGenerator' object.
generator = ImageDataGenerator(rescale=1/255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               horizontal_flip=True,
                               zoom_range=0.2)

In [ ]:
#This code defines a function Make_paths that creates a directory structure for training, validation, and testing data
#Making train , valid and test directories.
def Make_paths(base_path,labels)->str :#Contains the labels or classes for the dataset.
    '''function docstring'''
    
    if 'ready_to_gen' in listdir(base_path) :
        rmtree(f'{base_path}/ready_to_gen')
    mkdir(f'{base_path}/ready_to_gen')
    mkdir(f'{base_path}/ready_to_gen/train')
    mkdir(f'{base_path}/ready_to_gen/valid')
    mkdir(f'{base_path}/ready_to_gen/test')
    for Class in labels :
        if Class == 'ready_to_gen' or Class == 'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset' :
            continue
            
        mkdir(f'{base_path}/ready_to_gen/train/{Class}')
        mkdir(f'{base_path}/ready_to_gen/valid/{Class}')
        mkdir(f'{base_path}/ready_to_gen/test/{Class}')
    return f'{base_path}/ready_to_gen/train' , f'{base_path}/ready_to_gen/valid' , f'{base_path}/ready_to_gen/test'
path = "/kaggle/working/"
train , valid , test =  Make_paths(path,listdir("/kaggle/input/oral-diseases"))

In [ ]:
def split_data(photo_path,label_name,train_size)->None :
    '''function docstring'''
    size_tr = (train_size*len(listdir(photo_path)))//100
    train_select = sample(listdir(photo_path),size_tr)
    reamin_photos = [photo for photo in listdir(photo_path) if photo not in train_select]
    size_valid = len(reamin_photos)//2
    valid_select = sample(reamin_photos,size_valid)
    for pic in listdir(photo_path) :
        if pic in train_select :
            copyfile(f'{photo_path}/{pic}',f'{train}/{label_name}/{pic}')
        elif pic in valid_select :
            copyfile(f'{photo_path}/{pic}',f'{valid}/{label_name}/{pic}')
        else :
            copyfile(f'{photo_path}/{pic}',f'{test}/{label_name}/{pic}')

lbl = [l for l in listdir("/kaggle/input/oral-diseases") if l != 'ready_to_gen' and l != 'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset']
Addresses = ["/kaggle/input/oral-diseases/Data caries/Data caries/caries augmented data set/preview",
            "/kaggle/input/oral-diseases/Mouth Ulcer/Mouth Ulcer/Mouth_Ulcer_augmented_DataSet/preview",
            "/kaggle/input/oral-diseases/Tooth Discoloration/Tooth Discoloration /Tooth_discoloration_augmented_dataser/preview",
            "/kaggle/input/oral-diseases/hypodontia/hypodontia",
            "/kaggle/input/oral-diseases/Gingivitis/Gingivitis",
            "/kaggle/input/oral-diseases/Calculus/Calculus"]
for i in range(len(Addresses)) :
    split_data(Addresses[i],lbl[i],80)


In [ ]:
#Generating data. Data augmentation and generators
target_size = (155,155)
ready_train = generator.flow_from_directory(train,target_size=target_size,batch_size=20)
ready_valid = generator.flow_from_directory(valid,target_size=target_size,batch_size=11)
ready_test = generator.flow_from_directory(test,target_size=target_size,batch_size=11)

In [ ]:
pre = InceptionV3(include_top=False,input_shape=(255,255,3)) #This creates an instance of the InceptionV3 model from Keras' applications module, excluding the top (fully connected) layers
for layer in pre.layers :
        layer.trainable = False
last_l = pre.get_layer('mixed7')

In [ ]:
last_l.output

In [ ]:
#Creating a model by a function.
def Create_model(optm,Loss,acc)->None:
    '''function docstring'''
    pre = InceptionV3(include_top=False,input_shape=(155,155,3))
    for layer in pre.layers :
        layer.trainable = False
    last_l = pre.get_layer('mixed7')
    out = last_l.output
    x = layers.Flatten()(out)
    x = layers.Dense(1000,activation='relu')(x)
    x=layers.Normalization()(x)
    x = layers.Dense(255,activation='relu')(x)
    x=layers.Normalization()(x)
    x = layers.Dense(128,activation='relu')(x)
    x=layers.Normalization()(x)
    x = layers.Dense(64,activation='relu')(x)
    x=layers.Normalization()(x)
    x = layers.Dense(6,activation='softmax')(x)
    model = Model(pre.input,x)
    model.compile(optimizer='adam',
                 loss=Loss,
                 metrics=[acc])
    
    return model
alg = Create_model(RMSprop(learning_rate=0.0001),'categorical_crossentropy','accuracy')

In [ ]:
# alg.load_weights("/kaggle/working/weights")

# Assuming 'alg' is your model
weights_path = "/kaggle/working/weights.weights.h5"  # Change the file extension to .weights.h5
alg.save_weights(weights_path)

# Load weights
try:
    alg.load_weights(weights_path)
    print("Weights loaded successfully.")
except Exception as e:
    print("Error loading weights:", e)

In [ ]:
import tensorflow as tf
import numpy as np

def cosine_learning_rate_schedule(epoch, total_epochs, initial_lr):
    """
    Cosine learning rate schedule.
    
    Args:
        epoch (int): Current epoch number. number in the training process
        total_epochs (int): Total number of epochs.
        initial_lr (float): Initial learning rate.
    
    Returns:
        lr (float): Updated learning rate for the current epoch.
    """
    max_lr = initial_lr
    #Calculates the maximum and minimum learning rates.
    min_lr = 0.001 * initial_lr  # You can adjust this minimum LR as needed
    
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / total_epochs))
    lr = min_lr + 0.5 * (max_lr - min_lr) * cosine_decay
    
    return lr

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler #is a callback in TensorFlow Keras used to dynamically
#change the learning rate during training based on a specific function.

initial_learning_rate = 0.001  # You can adjust this initial LR as needed
total_epochs = 20  # Total number of epochs for training

# Create a learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lambda epoch: cosine_learning_rate_schedule(epoch, total_epochs, initial_learning_rate))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # You can choose a different metric like 'val_accuracy'
    patience=10,          # Number of epochs with no improvement after which training will be stopped.
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric.
)

In [ ]:
#this code evaluates a model's performance on a test set by computing accuracy,
#precision, and recall metrics, providing insight into how well the model performs in classification tasks.
from sklearn.metrics import precision_score, recall_score

# Assuming you have trained the model and stored the training history in 'history'

# Evaluate the model on the test set
results = alg.evaluate(ready_test)

# Predict the test set
predictions = alg.predict(ready_test)
predicted_classes = np.argmax(predictions, axis=1)

# Get true classes from the test set
true_classes = ready_test.classes

# Calculate precision and recall
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')

# Accuracy is available in the 'results' variable from model evaluation
accuracy = results[1]  # Assuming accuracy is the second metric

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [ ]:
#This setup allows for the evaluation of accuracy, precision, and recall on the validation set after each epoch during model training,
#providing insight into the model's performance and convergence
import numpy as np
from sklearn.metrics import precision_score, recall_score

class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        super(MetricsCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_pred = self.model.predict(self.validation_data[0])  # Validation images
        val_true = np.argmax(self.validation_data[1], axis=1)   # Validation labels
        val_pred_classes = np.argmax(val_pred, axis=1)

        val_acc = logs['val_accuracy']
        val_precision = precision_score(val_true, val_pred_classes, average='weighted')
        val_recall = recall_score(val_true, val_pred_classes, average='weighted')

        print(f'Epoch: {epoch + 1}, Validation Accuracy: {val_acc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}')

# Assuming you have defined and compiled your model (alg) and have ready_train and ready_valid datasets

# Get the validation data as numpy arrays
ready_valid_images, ready_valid_labels = next(iter(ready_valid))

# Instantiate the custom callback with validation data as numpy arrays
metrics_callback = MetricsCallback(validation_data=(ready_valid_images, ready_valid_labels))

# Train the model with custom callback
history = alg.fit(ready_train, callbacks=[metrics_callback, lr_scheduler], epochs=30, validation_data=ready_valid)


In [ ]:
#This code is used to visualize the performance of a neural network model during training. It plots two graphs
#Visualize models performance
epoch = range(1,len(history.epoch)+1)
results = history.history
plt.plot(epoch,results['accuracy'],'blue')
plt.plot(epoch,results['val_accuracy'],'red')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Oral_diseases')
plt.show()

plt.plot(epoch,results['loss'],'black')
plt.plot(epoch,results['val_loss'],'green')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Oral_diseases')
plt.show()

#These visualizations help in understanding the trends of accuracy and loss over the training epochs.
#It's common to observe how these metrics change over time to evaluate the model's performance

In [ ]:
#Evaluating data on test set
alg.evaluate(ready_test)

In [ ]:
import cv2
import requests


img_data = requests.get("https://assets.nhs.uk/nhsuk-cms/images/S_0118_mouth-ulcer_C0345376.width-1534.jpg").content
with open('image_name.jpg', 'wb') as handler:
   handler.write(img_data)

In [ ]:
dic={num:classes for classes ,num in ready_train.class_indices.items()}

In [ ]:
import cv2

image_path = r"/kaggle/input/oral-diseases/Tooth Discoloration/Tooth Discoloration /Tooth_discoloration_augmented_dataser/preview/Tooth_Discoloration_0_1.jpeg"
img = cv2.imread(image_path)#function is used to load an image from the specified path

if img is not None:  # Check if image loaded successfully
    img = cv2.resize(img, (155, 155))  # Resize the image is used to resize the image to a specified width and height (in this case, to a size of 155x155 pixels).
    img = img[..., ::-1] / 255  # Perform normalization or other operations
    # Further processing or use of the image
else:
    print("Failed to load the image at:", image_path)

In [ ]:
plt.imshow(img)


In [ ]:
#This code snippet checks if the variable i is an instance of either a NumPy array
import numpy as np

# Example array
i = np.array([5, 3, 8, 1, 2])

# Check if 'i' is an array or list
if isinstance(i, (np.ndarray, list)):
    sorted_indices = list(reversed(i.argsort()))
    print("Sorted Indices:", sorted_indices)
else:
    print("'i' is not an array or list.")

In [ ]:
#This code snippet seems to involve using a trained model (alg) to make predictions on an image represented by the variable img
for i in alg.predict(img[None]):
        print(dic[i.argmax(-1)])

In [ ]:
for i in alg.predict(img[None]):
   for o in list(reversed(i.argsort( ))):
        print(dic[o])

In [ ]:
! mkdir weights2

In [ ]:
alg.save_weights("/kaggle/working/weights.weights.h5")

In [ ]:
! mkdir model2

In [ ]:
alg.save("/kaggle/working/model2.h5")  # Save model in HDF5 format